In [ ]:
#STUDENT NAME: SHRUTHI
#STUDENT ID: 801218392
#HOMEWORK 3 QUESTION 2-B

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = 'C:/Users/Shruthi/Documents/HWs/RTML/hw2/data'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                            padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [6]:
class ResNet10(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
                    *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.dropout = nn.Dropout2d(p=0.3)
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)
   
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = self.dropout(out)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [7]:
model = ResNet10() 

In [8]:
import datetime

def training_loop_l2reg(n_epochs, optimizer, model, loss_fn,
                        train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            #imgs = imgs.to(device=device)
            #labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                          for p in model.parameters())  # <1>
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [9]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)  # <1>

optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop_l2reg(  # <5>
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-03-26 16:43:06.590779 Epoch 1, Training loss 1.7995484334123715
2022-03-26 16:58:58.924860 Epoch 10, Training loss 0.9947779642041686
2022-03-26 17:16:42.123520 Epoch 20, Training loss 0.8372888619942433
2022-03-26 17:34:30.658496 Epoch 30, Training loss 0.7557542784820737
2022-03-26 17:52:44.094720 Epoch 40, Training loss 0.7070414215859855
2022-03-26 18:10:16.738374 Epoch 50, Training loss 0.6651053291833614
2022-03-26 18:29:35.255554 Epoch 60, Training loss 0.6406844511361378
2022-03-26 18:47:18.420692 Epoch 70, Training loss 0.6158630244262383
2022-03-26 19:05:52.078059 Epoch 80, Training loss 0.60630457259505
2022-03-26 19:25:23.001104 Epoch 90, Training loss 0.5917978069132857
2022-03-26 19:45:34.890741 Epoch 100, Training loss 0.5779045486389218
2022-03-26 20:05:56.208153 Epoch 110, Training loss 0.5723788283212715
2022-03-26 20:26:16.989802 Epoch 120, Training loss 0.5641029179096222
2022-03-26 20:46:04.986748 Epoch 130, Training loss 0.567678845675705
2022-03-26 21:05:40.

In [10]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.85
Accuracy val: 0.73


In [11]:
import torchvision.models as models
from ptflops import get_model_complexity_info

net = model
macs, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

ResNet10(
  0.076 M, 100.000% Params, 0.025 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.001 M, 1.178% Params, 0.001 GMac, 3.709% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (resblocks): Sequential(
    0.009 M, 12.199% Params, 0.024 GMac, 96.025% MACs, 
    (0): ResBlock(
      0.009 M, 12.199% Params, 0.024 GMac, 96.025% MACs, 
      (conv): Conv2d(0.009 M, 12.115% Params, 0.024 GMac, 95.363% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.662% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      0.009 M, 12.199% Params, 0.024 GMac, 96.025% MACs, 
      (conv): Conv2d(0.009 M, 12.115% Params, 0.024 GMac, 95.363% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.662% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [12]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(76074, [864, 32, 9216, 32, 32, 65536, 32, 320, 10])